In [1]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render

from stackable_pipe import adapter, socket, ring
from solid import cube, rotate, translate

props = dict(d=60, w=4)
scene = rotate((0,0,60))(adapter(**props))
scene += rotate((0,0,-15))(translate((0,0,props['w']))(socket(**props,gap=0) ))
scene -= cube(100)

render(scene)


/home/gitpod/.pyenv/versions/3.8.12/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [27]:
from stackable_pipe import pipe, slot, ring, socket
from solid import cylinder, translate, rotate


def spin_around_tower(obj, count=4, alpha=45,h=160):
    angles = range(0, 360, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def tower_socket(d=60, w=4):
    r = d/2
    base = rotate((0, 0, 15))(socket(d=d, w=w)) + ring(d1=d, h=d+2*w, w=w/2)
    transition = ring(r2=r+w*3, r1=r, h=w*3, w=w*4) - cylinder(h=w*3, d=d)

    return base # + transition


def hydro_tower(w=4,h=160,d=160,pipe_count=5, alpha=45):
    tower = pipe(d=d, h=h, w=w)
    outline = cylinder(d=d, h=d, segments=60)
    holes = spin_around_tower(cylinder(d=60, h=h, segments=60), count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(translate((0, 0, d/2))(tower_socket(w=w)), h=d, count=pipe_count, alpha=alpha)
    
    holes = translate((0,0,-h/2))(holes)
    sockets = translate((0,0,-h/2))(sockets)

    hydro_tower = tower - holes
    hydro_tower += sockets
    hydro_tower *= outline
    return hydro_tower


render(hydro_tower(), outfile='hydro_tower.stl')


EXPORT-WARNING: Exported object may not be a valid 2-manifold and may need repair


In [3]:
from stackable_pipe import pipe, slot, ring, socket, adapter
from solid import cylinder, translate, rotate, cube

w=4
d = 60

bottom = adapter(d=d, w=w)
top = translate((0,0,4*w))(cylinder(d=d*1.2,h=5, segments=6))

hydro_tower_cap = top + bottom
hydro_tower_cap *= cube((d*2, d*2, 10*w), center=True)

# render(hydro_tower_cap, outfile='hydro_tower_cap.stl')

In [54]:
from stackable_pipe import pipe, slot, ring, socket, adapter
from solid import cylinder, translate, rotate, cube, hull

# def angle_cylinder(d=d, d2=d):
#     d2 = d2 or d
#     cyl = cylinder(d=d, h=2) + translate((d2/2,0,d2/4*3))(rotate((0,45,0))(cylinder(d=d, h=2)))
#     cyl = hull()(cyl)

#     return cyl

# def angle_pipe(d=d, w=w):
#     tube = angle_cylinder(d=d)
#     hole = angle_cylinder(d=d-2*w, d2=d+w)

#     return tube - hole


def adapter_bottom_grid(d=60, w=4):
    plate = cylinder(h=2, d=d-4*w)
    hole = cylinder(h=2, d=6)

    holes = sum([rotate((0, 0, r))(translate((x, 0, 0))(hole))
                for r in range(0, 360, 360//8) for x in [0, 9, 16]])

    return plate - holes


def hydro_tower_plant_cup(w=4, d=60, h=90, gap=0.1):
    bottom = adapter(d=d, w=w)
    grid = adapter_bottom_grid(d=d, w=w)
    pipe =  translate((0, 0, 5*w))(ring(d1=d-w, w=w, h=h))
    outline = cylinder(d=d-w+gap, h=h)

    return (bottom + grid + pipe) * outline


render(hydro_tower_plant_cup(), outfile='hydro_tower_plant_cup.stl')


In [56]:
d = 160
h=160
w=4
pipes=5
cups = spin_around_tower(translate((0,0, d/2))(hydro_tower_plant_cup(gap=0)), count=pipes)
cups = translate((0,0,-h/2))(cups)

render(hydro_tower(pipe_count=pipes) + cups - cube(200))

EXPORT-WARNING: Exported object may not be a valid 2-manifold and may need repair
